In [15]:
import string
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import  stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from difflib import SequenceMatcher
import warnings
warnings.filterwarnings('ignore')
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load a pre-trained Sentence Transformer model
model = SentenceTransformer('paraphrase-distilroberta-base-v1')



In [16]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\WB\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\WB\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\WB\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

**preprocessing**

In [17]:
data = pd.read_csv('./Statedriven.csv', encoding='unicode-escape')
data

,Requirement with Standard Syntax,Cleaned Data,sentence1,sentence2,similarity_score,Similarity
0,"When user register into the system, the system...","user register system, system shall allow user ...",user register system,system shall allow user select username passw...,0.395485,1
1,"while the game have 720p resolution, the game ...","game numbrp resolution, game shall playable",game numbrp resolution,game shall playable,0.358248,1
2,"When user access his/her account, the system s...","user access his/her account, system shall allo...",user access his/her account,system shall allow user enter password username.,0.410697,2
3,"When user play tournament, the system shall al...","user play tournament, system shall allow user ...",user play tournament,system shall allow user create team tournament.,0.548651,3
4,When user participate in the respective tourna...,"user participate respective tournament, system...",user participate respective tournament,system shall allow user select respective teams.,0.445075,2
5,When the player participate in the tournaments...,"player participate tournaments, system shall a...",player participate tournaments,system shall allow play one tournament,0.477092,3
6,"When the matches played, the system shall allo...","match played, system shall allow store informa...",match played,system shall allow store information match pl...,0.256208,5
7,"When the matches are being played, the system ...","match played, system shall allow stor informat...",match played,system shall allow stor information team matc...,0.334374,2
8,When the match played in respective touenament...,"match played respective touenament, system sha...",match played respective touenament,system shall store every outcome match.,0.248345,6
9,"When bowler have a specific bowler id, the sys...",bowler specific bowler id system shall allow s...,bowler specific bowler id system shall allow s...,bowler specific bowler id system shall allow s...,1.000000,7


In [18]:
data.columns
# printing all the columns headers

Index(['Requirement with Standard Syntax', 'Cleaned Data', 'sentence1',
       'sentence2', 'similarity_score', 'Similarity'],
      dtype='object')

In [19]:
data = data[['Requirement with Standard Syntax']]
data['Cleaned Data']=data['Requirement with Standard Syntax']
# selecting only the required columns

In [20]:
data
# printing the data

,Requirement with Standard Syntax,Cleaned Data
0,"When user register into the system, the system...","When user register into the system, the system..."
1,"while the game have 720p resolution, the game ...","while the game have 720p resolution, the game ..."
2,"When user access his/her account, the system s...","When user access his/her account, the system s..."
3,"When user play tournament, the system shall al...","When user play tournament, the system shall al..."
4,When user participate in the respective tourna...,When user participate in the respective tourna...
5,When the player participate in the tournaments...,When the player participate in the tournaments...
6,"When the matches played, the system shall allo...","When the matches played, the system shall allo..."
7,"When the matches are being played, the system ...","When the matches are being played, the system ..."
8,When the match played in respective touenament...,When the match played in respective touenament...
9,"When bowler have a specific bowler id, the sys...","When bowler have a specific bowler id, the sys..."


In [21]:
data.dropna(inplace=True)
# dropping the rows with null values in any column


In [31]:

# create a new column with null values
data['sentence1'] = data['Requirement with Standard Syntax'].isnull()
data['sentence2'] = data['Requirement with Standard Syntax'].isnull()
data['similarity_score'] = data['Requirement with Standard Syntax'].isnull()
data['SimilarSentence'] = data['Requirement with Standard Syntax'].isnull()

data


,Requirement with Standard Syntax,Cleaned Data,sentence1,sentence2,similarity_score,Similarity,POS,SimilarSentence
0,"When user register into the system, the system...","user register system, system shall allow user ...",False,False,False,1.0,"[(When, WRB), (user, JJ), (register, NN), (int...",False
1,"while the game have 720p resolution, the game ...","game numbrp resolution, game shall playable",False,False,False,1.0,"[(while, IN), (the, DT), (game, NN), (have, VB...",False
2,"When user access his/her account, the system s...","user access his/her account, system shall allo...",False,False,False,2.0,"[(When, WRB), (user, JJ), (access, NN), (his/h...",False
3,"When user play tournament, the system shall al...","user play tournament, system shall allow user ...",False,False,False,4.0,"[(When, WRB), (user, JJ), (play, NN), (tournam...",False
4,When user participate in the respective tourna...,"user participate respective tournament, system...",False,False,False,4.0,"[(When, WRB), (user, JJ), (participate, NN), (...",False
5,When the player participate in the tournaments...,"player participate tournaments, system shall a...",False,False,False,2.0,"[(When, WRB), (the, DT), (player, NN), (partic...",False
6,"When the matches played, the system shall allo...","match played, system shall allow store informa...",False,False,False,3.0,"[(When, WRB), (the, DT), (matches, NNS), (play...",False
7,"When the matches are being played, the system ...","match played, system shall allow stor informat...",False,False,False,6.0,"[(When, WRB), (the, DT), (matches, NNS), (are,...",False
8,When the match played in respective touenament...,"match played respective touenament, system sha...",False,False,False,5.0,"[(When, WRB), (the, DT), (match, NN), (played,...",False
9,"When bowler have a specific bowler id, the sys...","bowler specific bowler id, system shall allow ...",False,False,False,1.0,"[(When, WRB), (bowler, NN), (have, VBP), (a, D...",False


In [23]:
data['Cleaned Data'] = data['Cleaned Data'].str.lower()
data['Cleaned Data'] = data['Cleaned Data'].str.replace(r'^.+@[^\.].*\.[a-z]{2,}$', 'emailaddress')
data['Cleaned Data'] = data['Cleaned Data'].str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$', 'webaddress')
data['Cleaned Data'] = data['Cleaned Data'].str.replace(r'£|\$', 'dollers')
data['Cleaned Data'] = data['Cleaned Data'].str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$', 'phonenumber')
data['Cleaned Data'] = data['Cleaned Data'].str.replace(r'\d+(\.\d+)?', 'numbr')
data['Cleaned Data'] = data['Cleaned Data'].apply(lambda x: ' '.join(term for term in x.split() if term not in string.punctuation))
stop_words = set(stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure'])
data['Cleaned Data'] = data['Cleaned Data'].apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))
lemmatizer = WordNetLemmatizer()
data['Cleaned Data'] = data['Cleaned Data'].apply(lambda x: ' '.join(lemmatizer.lemmatize(t) for t in x.split()))

data['Cleaned Data']


0     user register system, system shall allow user ...
1           game numbrp resolution, game shall playable
2     user access his/her account, system shall allo...
3     user play tournament, system shall allow user ...
4     user participate respective tournament, system...
5     player participate tournaments, system shall a...
6     match played, system shall allow store informa...
7     match played, system shall allow stor informat...
8     match played respective touenament, system sha...
9     bowler specific bowler id, system shall allow ...
10          user select level, system shall start game.
11    user click option button, system shall take us...
12    ai traffic added, system shall use ai vehicle ...
13    specified rule violated, system shall reduce p...
14    user clear previous level, game shall alllow u...
15    user click start button main menu, system shal...
16    user select vehicle, system shall display sele...
17    user press exit button, system shall allow

In [34]:

def calculate_similarity_score(sentence1, sentence2, model):
    # Generate embeddings for both sentences
    embeddings = model.encode([sentence1, sentence2])

    # Compute cosine similarity between the embeddings
    similarity = cosine_similarity(embeddings[0].reshape(1, -1), embeddings[1].reshape(1, -1))

    # Extract the similarity score (a value between -1 and 1)
    similarity_score = similarity.item()

    return similarity_score

def compareWithall(sentence1):
    sim=0
    rowCount = 0
    rownumber = 0
    #compare the sentence with all the sentences in sentence2 column
    for index, row in data.iterrows():
        similarity= calculate_similarity_score(sentence1, row['sentence2'], model)
        if similarity > 0.2:
            sim = sim + 1
            rownumber = rownumber + 1
            data.at[index, 'Similarity'] = sim
            rowCount = rowCount + 1
            data.at[rownumber, sentence1] = "If " + sentence1 + " then " + row['sentence2']
            
         


# iterate through the rows of the data frame Requirement with Standard Syntax
for index, row in data.iterrows():
    #split the sentence if it contains a full a comma
    if ',' in row['Cleaned Data']:
        sentence1 = row['Cleaned Data'].split(',')[0]
        sentence2 = row['Cleaned Data'].split(',')[1]
        data.at[index, 'sentence1'] = sentence1
        data.at[index, 'sentence2'] = sentence2
        data.at[index, 'similarity_score'] = calculate_similarity_score(sentence1, sentence2, model)
    else:
        data.at[index, 'sentence1'] = row['Cleaned Data']
        data.at[index, 'sentence2'] = row['Cleaned Data']
        
        data.at[index, 'similarity_score'] = calculate_similarity_score(row['Cleaned Data'], row['Cleaned Data'], model)

for index, row in data.iterrows():
    compareWithall(row['sentence1'])
   




#writing the data to a csv file
data.to_csv('Statedriven.csv', index=False)


In [25]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\WB\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [26]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
# punkt is a pre-trained tokenizer for english language and averaged_perceptron_tagger is a pre-trained part-of-speech tagger for english language

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\WB\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\WB\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [27]:
from collections import Counter
from nltk import word_tokenize, pos_tag, pos_tag_sents
texts = data['Requirement with Standard Syntax'].tolist()
tagged_texts = pos_tag_sents(map(word_tokenize, texts))


data['POS'] = tagged_texts
data

,Requirement with Standard Syntax,Cleaned Data,sentence1,sentence2,similarity_score,Similarity,POS
0,"When user register into the system, the system...","user register system, system shall allow user ...",user register system,system shall allow user select username passw...,0.395485,1.0,"[(When, WRB), (user, JJ), (register, NN), (int..."
1,"while the game have 720p resolution, the game ...","game numbrp resolution, game shall playable",game numbrp resolution,game shall playable,0.358248,1.0,"[(while, IN), (the, DT), (game, NN), (have, VB..."
2,"When user access his/her account, the system s...","user access his/her account, system shall allo...",user access his/her account,system shall allow user enter password username.,0.410697,2.0,"[(When, WRB), (user, JJ), (access, NN), (his/h..."
3,"When user play tournament, the system shall al...","user play tournament, system shall allow user ...",user play tournament,system shall allow user create team tournament.,0.548651,2.0,"[(When, WRB), (user, JJ), (play, NN), (tournam..."
4,When user participate in the respective tourna...,"user participate respective tournament, system...",user participate respective tournament,system shall allow user select respective teams.,0.445075,1.0,"[(When, WRB), (user, JJ), (participate, NN), (..."
5,When the player participate in the tournaments...,"player participate tournaments, system shall a...",player participate tournaments,system shall allow play one tournament,0.477092,3.0,"[(When, WRB), (the, DT), (player, NN), (partic..."
6,"When the matches played, the system shall allo...","match played, system shall allow store informa...",match played,system shall allow store information match pl...,0.256208,4.0,"[(When, WRB), (the, DT), (matches, NNS), (play..."
7,"When the matches are being played, the system ...","match played, system shall allow stor informat...",match played,system shall allow stor information team matc...,0.334374,2.0,"[(When, WRB), (the, DT), (matches, NNS), (are,..."
8,When the match played in respective touenament...,"match played respective touenament, system sha...",match played respective touenament,system shall store every outcome match.,0.248345,7.0,"[(When, WRB), (the, DT), (match, NN), (played,..."
9,"When bowler have a specific bowler id, the sys...","bowler specific bowler id, system shall allow ...",bowler specific bowler id,system shall allow store information bowler.,0.338708,1.0,"[(When, WRB), (bowler, NN), (have, VBP), (a, D..."
